# Tutorial-Compressor efficiency map generation

This tutorial explains the steps to be followed to generate the series of compressor
efficiency ('eta_s') values for the design point data of a particular heat pump model.
In this example, the data for "Air_30kW" model has been used.


### Parametrizing the model for the nominal capacity data

Following the advice given in the documentation, the different components of the network
have to be parametrized such that the compressor power calculated by the model matches that
of the data sheet, for the nominal capacity data. For the heat pump model in this example, the
heat load is 16.3 kW & the power required is 3.83 kW, at an ambient air temperature of 7 deg. C
and heat supply temperature of 35 deg. C.

Code: 'Parametrization_NominalData.py'

The network solution for this case should be saved as it will be used in the following steps.
The network is saved as 'Par_NominalData'.


### Setting up the input data files

The data from the heating capacity tables should be available in csv format.
The heating capacities (in kW) and power requirement (in kW) at the different evaporator
and condenser temperatures should be available as shown in the 'Data\Heat_Load_Data.csv' and
'Data\PI_Data.csv' respectively. The first row contains the condenser leaving temperatures
and the first column contains the ambient temperature. The temperatures are in deg. C.


In [ ]:
from tespy.networks import load_network
import pandas as pd
import numpy as np
from tqdm import tqdm
import json

In [ ]:
# import the saved network with the initial parametrization data

imported_plant = load_network('.\Par_NominalData')
# imported_plant.solve('design')
# imported_plant.print_results()

In [ ]:
#import the heat load data

HL_Data_Path = '.\Data\Heat_Load_Data.csv'
heat_load_data = pd.read_csv(HL_Data_Path, header=0, delimiter=',')
print(heat_load_data)

In [ ]:
#import the power requirement data

PI_Data_Path = '.\Data\PI_Data.csv'
PI_data = pd.read_csv(PI_Data_Path, header=0, delimiter=',')
print(PI_data)


In [ ]:
# A DataFrame to save the power calculated by the model

PI_Data_new= pd.DataFrame().reindex_like(PI_data)
PI_Data_new['heat_source_T'] = PI_data['heat_source_T']
print( PI_Data_new)

In [ ]:
# The Leaving Water Condenser (LWC) temperatures
cols = heat_load_data.columns[1:]
print(cols)

In [ ]:
# The range of values for the efficiency of the compressor from which the best
# value is chosen
etas_list = np.arange(0.25,0.96,0.01)

In [ ]:
# A DataFrame to save the best efficiency values identified for each design point
etas_data = pd.DataFrame().reindex_like(heat_load_data)
etas_data['heat_source_T'] = heat_load_data['heat_source_T']
print(etas_data)

In [ ]:
total = len(etas_data.index) * len(cols)

In [ ]:
pbar = tqdm(total=total)
for i in range(len(etas_data.index)):
    heat_source_T = heat_load_data['heat_source_T'][i] # The ambient temperature temperature
    k = 30 # index for moving within etas_list
    for j in cols:
        LWC = float(j) # The Leaving Water Condenser (LWC) temperature
        HL = float(heat_load_data[j][i]*1000) # Heat Load in W

        l=1
        if  not np.isnan(HL):
            # print ('row: %d , column: %s'%(i,j))
            # The display of iteration info during network solving is turned off
            imported_plant.set_attr(iterinfo=False)

            # A temperature difference of 5 deg.C is assumed across the condenser
            imported_plant.get_conn('condenser:out2_consumer:in1').T.val= LWC
            imported_plant.get_conn('consumer cycle closer:out1_condenser recirculation pump:in1').T.val =LWC-5

            # A temperature difference of 5 deg.C is assumed across the evaporator
            imported_plant.get_conn('source ambient:out1_ambient pump:in1').T.val=heat_source_T
            imported_plant.get_conn('evaporator:out1_sink ambient:in1').T.val= heat_source_T - 5

            # The heat load is set
            imported_plant.get_comp('consumer').set_attr(Q=-HL)

            # print("row: %d, column: %s" %(i,j))

            # The efficiency value for the compressor that gives the power closest
            # to the power from the data sheet is found
            while 0 <= k < len(etas_list):

                imported_plant.get_comp('compressor').eta_s.val = etas_list[k]
                imported_plant.solve('design')

                P_new = imported_plant.get_comp('compressor').P.val/1000

                P_new_dif = P_new - PI_data.loc[i, j]

                if abs(P_new_dif) > 0.15 or k==(len(etas_list) - 1):

                    if P_new_dif < 0:
                        k -= 1
                    elif k == (len(etas_list) - 1):
                        PI_Data_new.loc[i,j] = P_new
                        etas_data.loc[i,j] = etas_list[k]
                        break
                    else:
                        k += 1

                elif abs(P_new_dif) <= 0.15 and P_new_dif < 0:
                    # print("entering 2nd condition: %d time" %l)
                    k -= 1
                    P_old = P_new
                    P_old_dif = P_new_dif
                    # print(P_old, P_old_dif)
                    imported_plant.get_comp('compressor').eta_s.val = etas_list[k]
                    imported_plant.solve('design')
                    P_new = imported_plant.get_comp('compressor').P.val/1000
                    P_new_dif = P_new - PI_data.loc[i, j]
                    # print(P_new, P_new_dif)
                    if round(abs(P_new_dif),2) <= round(abs(P_old_dif),2):
                        # print("entering 2-1 condition")
                        PI_Data_new.loc[i,j] = P_new
                        etas_data.loc[i,j] = etas_list[k]
                    else:
                        # print("entering 2-2 condition")
                        k += 1
                        PI_Data_new.loc[i,j] = P_old
                        etas_data.loc[i,j] = etas_list[k]
                    # print(PI_Data_new[j][i], etas_data[j][i])
                    break

                elif abs(P_new_dif)<=0.15 and P_new_dif > 0:
                    # print("entering 3rd condition: %d time" %l)
                    k += 1
                    P_old = P_new
                    P_old_dif = P_new_dif
                    imported_plant.get_comp('compressor').eta_s.val = etas_list[k]
                    imported_plant.solve('design')
                    P_new = imported_plant.get_comp('compressor').P.val/1000
                    P_new_dif = P_new - PI_data.loc[i,j]
                    if round(abs(P_new_dif),2) < round(abs(P_old_dif),2):
                        PI_Data_new.loc[i,j] = P_new
                        etas_data.loc[i,j] = etas_list[k]
                    else:
                        k -= 1
                        PI_Data_new.loc[i,j] = P_old
                        etas_data.loc[i,j] = etas_list[k]
                    # print(PI_Data_new[j][i], etas_data[j][i])
                    break

        else:
            etas_data.loc[i,j] = np.nan

        pbar.update()

pbar.close()

In [ ]:
print(etas_data)


In [ ]:
etas_data.to_csv('parametrization_etas.csv')

In [ ]:
print(PI_Data_new - PI_data)

In [ ]:
#Convert it to json? or some other type?

In [ ]:
etas_data.set_index('heat_source_T', inplace=True)
result = etas_data.to_json(orient="index")
parsed = json.loads(result)
json.dumps(parsed, indent=4)

In [ ]:
data={}
data['eta_s'] = parsed

In [ ]:
heat_load_data.set_index('heat_source_T', inplace=True)
result = heat_load_data.to_json(orient="index")
parsed = json.loads(result)
json.dumps(parsed, indent=4)
data['heatload'] = parsed

In [ ]:
ids = np.arange(1,etas_data.size+1,1)
ids_x = [ids[x:x+len(etas_data.columns)] for x in range(0,len(ids), len(etas_data.columns)) ]
id_list = pd.DataFrame(ids_x, columns=etas_data.columns, index=etas_data.index)
result = id_list.to_json(orient="index")
parsed = json.loads(result)
json.dumps(parsed, indent=4)
data['ids'] = parsed

In [ ]:
# The minimum heatload for this model of heat pump should be provided
data['min_heatload'] = 15600
data['max_heatload'] = 1000* heat_load_data.max().max()

In [ ]:
# Give a name to the model of heat pump being parametrized
model_name = 'Air_30kW'
with open("../../mosaik_components/heatpump/eta_s_data.json") as feedsjson:
    feeds = json.load(feedsjson)

if model_name in feeds.keys():
    a = input('The model name already exists and must be replaced. Do you want to replace the data (y/n)? : ')
    if a.lower() == 'y':
        feeds[model_name]= data
    elif a.lower() =='n':
        model_name = input('Enter new model name')
        feeds[model_name]= data
    else:
        print("Wrong input. Execute cell again.")
else:
    feeds[model_name] = data

In [ ]:
with open("../../mosaik_components/heatpump/eta_s_data.json", "w") as write_file:
    json.dump(feeds, write_file, indent=4)